In [22]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm 

In [23]:
# Var : Variable
# Relas : Relational (sama dengan,tidak sama dengan,lebih dari, kurang dari, lebih dari sama dengan, kurang dari sama dengan)
# Gate : Logical Gate

TRAIN_DATA = [
    ('Total Buah lebih dari 15', { #lebih dari
        'entities': [(11, 21, 'Relas')]
    }),
     ('Cable Type sama dengan Optic', { #sama dengan
        'entities': [(11, 22, 'Relas')]
    }),
    ('Tanggal kurang dari 17', { #kurang dari
        'entities': [(8, 19, 'Relas')]
    }),
    ('Username sama dengan benar dan Password sama dengan benar', { #sama dengan, dan, sama dengan
        'entities': [(9, 20, 'Relas'), (27, 30, 'Gate'), (40, 51, 'Relas')]
    }),
    ('Harga lebih dari 1000 dan Total kurang dari 2000', {
        'entities': [(6, 16, 'Relas'), (22, 25, 'Gate'), (32, 43, 'Relas')] #lebih dari, dan, kurang dari
    }),
    ('Rating lebih dari sama dengan 4 dan Harga kurang dari 500', { #lebih dari sama dengan, dan, kurang dari
        'entities': [(7, 29, 'Relas'), (32, 35, 'Gate'), (42, 53, 'Relas')]
    }),
    ('Suhu sama dengan 25 atau Kelembaban lebih dari 70', { #sama dengan, atau, lebih dari
        'entities': [(5, 16, 'Relas'), (20, 24, 'Gate'), (36, 46, 'Relas')]
    }),
    ('Juara sama dengan 1', { #sama dengan
        'entities': [(6, 17, 'Relas')]
    }),
    ('Jarak lebih dari sama dengan 300 meter dan Kecepatan lebih dari 25', { #lebih dari sama dengan, dan, lebih dari
        'entities': [(6, 28, 'Relas'), (39, 42, 'Gate'), (53, 63, 'Relas')]
    }),
    ('Berat tidak sama dengan 100 kg', { #tidak sama dengan
        'entities': [(6, 23, 'Relas')]
    })
]

In [24]:
model = None
output_dir=Path("/home/pradeva/Documents/NER-API/MODEL")
n_iter=100

In [25]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


In [26]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

ner

In [27]:
from spacy.training.example import Example

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

correct_train_entities = 0  # Untuk menghitung entitas yang diidentifikasi dengan benar pada data pelatihan
total_train_entities = 0    # Untuk menghitung total entitas sebenarnya pada data pelatihan

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update(
                [example],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
            
         # Hitung akurasi selama pelatihan
        doc = nlp(text)
        for ent in annotations["entities"]:
            start, end, label = ent
            if (start, end, label) in [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]:
                correct_train_entities += 1
            total_train_entities += 1
        
        accuracy_train = (correct_train_entities / total_train_entities) * 100
        print(f"Iterasi {itn + 1}, Train Data Accuracy: {accuracy_train:.2f}%")

        print(losses)


100%|██████████| 10/10 [00:00<00:00, 75.86it/s]


Iterasi 1, Train Data Accuracy: 0.00%
{'ner': 57.21972641348839}


100%|██████████| 10/10 [00:00<00:00, 78.33it/s]


Iterasi 2, Train Data Accuracy: 0.00%
{'ner': 34.44277781248093}


100%|██████████| 10/10 [00:00<00:00, 82.07it/s]


...

Iterasi 98, Train Data Accuracy: 89.89%
{'ner': 1.3783125206575135}


100%|██████████| 10/10 [00:00<00:00, 69.19it/s]


Iterasi 99, Train Data Accuracy: 90.06%
{'ner': 2.3675536243687163e-06}


100%|██████████| 10/10 [00:00<00:00, 69.39it/s]

Iterasi 100, Train Data Accuracy: 90.22%
{'ner': 0.0007119634434587432}


In [28]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob_) for t in doc])

Entities [('tidak sama dengan', 'Relas')]
Tokens [('Berat', '', 'O'), ('tidak', 'Relas', 'B'), ('sama', 'Relas', 'I'), ('dengan', 'Relas', 'I'), ('100', '', 'O'), ('kg', '', 'O')]
Entities [('lebih dari', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Tokens [('Harga', '', 'O'), ('lebih', 'Relas', 'B'), ('dari', 'Relas', 'I'), ('1000', '', 'O'), ('dan', 'Gate', 'B'), ('Total', '', 'O'), ('kurang', 'Relas', 'B'), ('dari', 'Relas', 'I'), ('2000', '', 'O')]
Entities [('sama dengan', 'Relas'), ('atau', 'Gate'), ('lebih dari', 'Relas')]
Tokens [('Suhu', '', 'O'), ('sama', 'Relas', 'B'), ('dengan', 'Relas', 'I'), ('25', '', 'O'), ('atau', 'Gate', 'B'), ('Kelembaban', '', 'O'), ('lebih', 'Relas', 'B'), ('dari', 'Relas', 'I'), ('70', '', 'O')]
Entities [('sama dengan', 'Relas'), ('dan', 'Gate'), ('sama dengan', 'Relas')]
Tokens [('Username', '', 'O'), ('sama', 'Relas', 'B'), ('dengan', 'Relas', 'I'), ('benar', '', 'O'), ('dan', 'Gate', 'B'), ('Password', '', 'O'), ('sama', 'Relas', 'B'),

In [29]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)    

Saved model to /home/pradeva/Documents/NER-API/MODEL


In [30]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from /home/pradeva/Documents/NER-API/MODEL
Entities [('tidak sama dengan', 'Relas')]
Entities [('lebih dari', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Entities [('sama dengan', 'Relas'), ('atau', 'Gate'), ('lebih dari', 'Relas')]
Entities [('sama dengan', 'Relas'), ('dan', 'Gate'), ('sama dengan', 'Relas')]
Entities [('lebih dari sama dengan', 'Relas'), ('dan', 'Gate'), ('kurang dari', 'Relas')]
Entities [('lebih dari', 'Relas')]
Entities [('sama dengan', 'Relas')]
Entities [('kurang dari', 'Relas')]
Entities [('sama dengan', 'Relas')]
Entities [('lebih dari sama dengan', 'Relas'), ('dan', 'Gate'), ('lebih dari', 'Relas')]


In [10]:
#TEST -----------------------------------------------------------

In [31]:
# Load the trained NER model
nlp = spacy.load(output_dir)

# Test text
test_text = "Total_buah sama dengan 17"

# Process the test text
doc = nlp(test_text)


# Print extracted entities
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob_) for t in doc])

Entities [('sama dengan', 'Relas')]
Tokens [('Total_buah', '', 'O'), ('sama', 'Relas', 'B'), ('dengan', 'Relas', 'I'), ('17', '', 'O')]


In [32]:
# Data tes
TEST_DATA = [
    ("Total_buah sama dengan 17 dan besar bola kurang dari 17", {
        "entities": [(0, 10, "Var"), (11, 22, "Relas"), (26, 29, "Gate"), (30, 40, "Var"), (41, 52, "Relas")]
    }),
    ("Suara sama dengan 100 dan juara sama dengan menang", {
        "entities": [(0, 5, "Var"), (6, 17, "Relas"), (22, 25, "Gate"), (26, 31, "Var"), (32, 43, "Relas")]
    }),
    ("apel kurang dari 10 dan harga_beras lebih dari 200", {
        "entities": [(0, 4, "Var"), (5, 16, "Relas"), (20, 23, "Gate"), (24, 35, "Var"), (36, 46, "Relas")]
    })
    # Tambahkan data tes lainnya sesuai kebutuhan
]

correct_entities = 0  # Untuk menghitung entitas yang diidentifikasi dengan benar
total_entities = 0   # Untuk menghitung total entitas sebenarnya

for text, annotations in TEST_DATA:
    print(text)
    print(annotations)
    doc = nlp(text)  # Proses data tes dengan model
    print(doc.ents)

    for ent in doc.ents:
        print(ent.text, ent.label_)
    
    for ent in annotations["entities"]:
        start, end, label = ent
        if (start, end, label) in [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]:
            correct_entities += 1
        print(start, end, label)
        print(ent)
        total_entities += 1

accuracy = (correct_entities / total_entities) * 100
print(f"Accuracy: {accuracy:.2f}%")

Total_buah sama dengan 17 dan besar bola kurang dari 17
{'entities': [(0, 10, 'Var'), (11, 22, 'Relas'), (26, 29, 'Gate'), (30, 40, 'Var'), (41, 52, 'Relas')]}
(sama dengan, dan, kurang dari)
sama dengan Relas
dan Gate
kurang dari Relas
0 10 Var
(0, 10, 'Var')
11 22 Relas
(11, 22, 'Relas')
26 29 Gate
(26, 29, 'Gate')
30 40 Var
(30, 40, 'Var')
41 52 Relas
(41, 52, 'Relas')
Suara sama dengan 100 dan juara sama dengan menang
{'entities': [(0, 5, 'Var'), (6, 17, 'Relas'), (22, 25, 'Gate'), (26, 31, 'Var'), (32, 43, 'Relas')]}
(sama dengan, dan, sama dengan)
sama dengan Relas
dan Gate
sama dengan Relas
0 5 Var
(0, 5, 'Var')
6 17 Relas
(6, 17, 'Relas')
22 25 Gate
(22, 25, 'Gate')
26 31 Var
(26, 31, 'Var')
32 43 Relas
(32, 43, 'Relas')
apel kurang dari 10 dan harga_beras lebih dari 200
{'entities': [(0, 4, 'Var'), (5, 16, 'Relas'), (20, 23, 'Gate'), (24, 35, 'Var'), (36, 46, 'Relas')]}
(kurang dari, dan, lebih dari)
kurang dari Relas
dan Gate
lebih dari Relas
0 4 Var
(0, 4, 'Var')
5 16 Relas
